#Exploratory Data Analysis on the landplots

# 0. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
feature_frame=pd.read_parquet('muestreos_parcelas.parquet')
feature_frame

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada
0,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-16,2006,23,109,0,0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-27,2006,23,109,0,0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-04,2006,23,109,0,0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-11,2006,23,109,0,0,NaN,NaN,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-17,2006,23,109,0,0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581788,2021-04-05 17:26:06,008-00013-01-2,sevilla,villanueva de san juan,2021-03-02,2021,8,13,1,2,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-05 17:24:32,303039.00,4104305.00,570.0,LA SIERRA SUR,5.58,18.0,NaN,Franco-arcilloso,Secano,No,NaN,NaN,8 X 8,156.0,2.0,NaN,NaN,HOJIBLANCO,01/03/1990,NaN,NaN,NaN,LAS ATALAYAS,SE/OL/04 SIERRA SUR (OESTE),El Saucejo,El Saucejo,NaN,LAS ATALAYAS,NaN,NaN,NaN,NaN,NaN,SI,Producción Integrada (PI)
581789,2021-04-05 17:26:06,008-0

# 1. Filtering of rows

Taking into account filters of:
- Removing codparcelas null values
- Those that have next measurement inside X days
- Those that the status of the olive does not descend

In [4]:
def find_estado_with_value_two(row) -> int:
    """
    Returns numero del siguiente etado (yt+1). (e.g., 13)
    """
    for column in row.index:
        if row[column] == 2:
            number_growth_stage = int(column.split("_")[-1])
            return number_growth_stage


def get_valid_dataset(df: pd.DataFrame(), max_days_till_next_date: int) -> pd.DataFrame():
    estados = [
    "estado_fenologico_1",
    "estado_fenologico_2",
    "estado_fenologico_3",
    "estado_fenologico_4",
    "estado_fenologico_5",
    "estado_fenologico_6",
    "estado_fenologico_7",
    "estado_fenologico_8",
    "estado_fenologico_9",
    "estado_fenologico_10",
    "estado_fenologico_11",
    "estado_fenologico_12",
    "estado_fenologico_13",
    "estado_fenologico_14",
    ]
    # Removing null and filling empty estados with 0
    df = df.dropna(subset=estados, how="all")
    df = df.dropna(subset=["codparcela"])
    df[estados] = df[estados].fillna(0)

    #Filtering Dataset to keep rows only with one unique 2
    df["count_2s"] = df[estados].eq(2).sum(axis=1)
    df = df[df.count_2s == 1]

    # Sorting by date
    df["fecha"] = pd.to_datetime(df["fecha"])
    df.sort_values(by="fecha", inplace=True)

    # Creating a column to display the number of days till the next observation
    df["next_date"] = df.groupby("codparcela", observed=True)["fecha"].shift(-1)
    df["days_until_next_visit"] = (df["next_date"] - df["fecha"]).dt.days

    # Creating a column to display the next estado_fenológico (yt+1)
    df["estado_actual"] = df[estados].apply(find_estado_with_value_two, axis=1)
    df["next_estado"] = df.groupby("codparcela", observed=True)["estado_actual"].shift(-1)

    # Removing the parcels with only one entry and the last entry for every parcel
    df = df.dropna(subset=["days_until_next_visit"])  # 5150 entries removed

    #Removing entries where estado decreases
    mask_estado_decrease = df["next_estado"] - df["estado_actual"] < 0
    df = df[~mask_estado_decrease]

    #Changing datatypes
    df["days_until_next_visit"] = df["days_until_next_visit"].astype("int16")
    df["next_estado"] = df["next_estado"].astype("int8")
    df[estados] = df[estados].astype("int8")

    # Filtering the max days
    df = df[df["days_until_next_visit"] < max_days_till_next_date]

    excluded_columns = ['estado_actual','count_2s', 'next_date']

    return df.loc[:, [i for i in df.columns if i not in excluded_columns]]

In [5]:
feature_frame=get_valid_dataset(feature_frame, 30)
print(len(feature_frame))

536444


# 2. Analysis of variables

1. Generated_muestreos

In [6]:
#Variable that indicates the time when the measuring was introduced in the system/database, NOT when the measurement was done.
#So, only during two days was all the data introduced in the database: 2020-04-23 and 2021-04-05.
#This variable is, in principle, irrelevant.

print(feature_frame["generated_muestreos"].isnull().sum())
numvalues_generatedmuestreos = len(set(feature_frame["generated_muestreos"]))
print(numvalues_generatedmuestreos)
counter_generatedmuestreos=Counter(feature_frame["generated_muestreos"])
print(counter_generatedmuestreos)

0
8
Counter({Timestamp('2021-04-05 17:26:06'): 147396, Timestamp('2020-04-23 16:54:05'): 114644, Timestamp('2020-04-23 14:34:36'): 104216, Timestamp('2020-04-23 14:39:25'): 98597, Timestamp('2020-04-23 17:00:15'): 49439, Timestamp('2020-04-23 14:31:50'): 13685, Timestamp('2020-04-23 14:42:09'): 5928, Timestamp('2020-04-23 14:30:39'): 2539})


2. Codparcela

In [7]:
#Variable that indicates the codes of the different landplots.
#There is a total of 5239 landplots.
#Not all of the landplots are measured equally; some of them have many measurements, some others have less.

#Some landplots may be very big (many measurements in 1 landplot). This should not be the case because in the code
#some classification of sectors and subsectors is developed.

#Therefore, for some tbd reason some of them may have more time measurements than others.

#This variable is, in principle, irrelevant, if needed there is a splitting of the variable in other ones.

print(feature_frame["codparcela"].isnull().sum())
numvalues_codparcela = len(set(feature_frame["codparcela"]))
print(numvalues_codparcela)
counter_codparcela=Counter(feature_frame["codparcela"])
print(counter_codparcela)

0
5061
Counter({'015-00002-00-00': 736, '043-00008-00-00': 697, '017-00204-00-00': 670, '010-00016-00-00': 631, '004-00001-00-00': 604, '004-00009-00-00': 579, '007-00018-01-01': 579, '012-00001-00-00': 568, '006-00001-00-00': 541, '009-00006-00-00': 538, '004-00017-00-00': 492, '032-00060-00-00': 490, '010-00201-00-00': 489, '011-00021-00-00': 481, '043-00145-00-00': 480, '026-00192-00-00': 477, '017-00291-00-00': 475, '043-00190-00-00': 475, '039-00017-00-00': 475, '018-00022-00-00': 473, '004-00010-00-00': 468, '051-00015-00-00': 467, '074-00035-00-00': 466, '009-00151-00-00': 465, '034-00051-00-00': 462, '008-00029-00-00': 437, '010-00031-00-00': 437, '014-00004-00-00': 436, '033-00069-00--1': 435, '005-00013-00-00': 434, '027-00188-00--1': 434, '007-00034-00-00': 433, '012-00207-00-00': 433, '007-00022-00-00': 432, '003-00040-00-00': 432, '006-00658-00-00': 431, '024-00189-00-00': 431, '025-00209-00--1': 428, '013-00003-00-00': 428, '001-00101-00-00': 428, '018-00040-00-00': 428, 

In [8]:
pd.DataFrame.from_dict(counter_codparcela.values()).describe()

,0
count,5061.000000
mean,105.995653
std,98.577397
min,1.000000
25%,32.000000
50%,73.000000
75%,142.000000
max,736.000000


3. Provincia

In [9]:
#Variable that identifies the region in Spain where the landplot is.
#This variable is, in principle, irrelevant, as there is more precise information regarding the specific location of the landplots.

print(feature_frame["provincia"].isnull().sum())
counter_provincia=Counter(feature_frame["provincia"])
print(counter_provincia)

0
Counter({'jaen': 149374, 'granada': 127952, 'cordoba': 126910, 'malaga': 61868, 'sevilla': 41768, 'cadiz': 18681, 'huelva': 7352, 'almeria': 2539})


In [10]:
percentage_provincia = {key: round((value / sum(counter_provincia.values())*100),2) for key,value in counter_provincia.items()}
print(percentage_provincia)

{'malaga': 11.53, 'cordoba': 23.66, 'jaen': 27.85, 'granada': 23.85, 'cadiz': 3.48, 'huelva': 1.37, 'almeria': 0.47, 'sevilla': 7.79}


In [11]:
unique_parcelas_provincia=feature_frame.drop_duplicates(subset="codparcela")
counter_provincia2=Counter(unique_parcelas_provincia["provincia"])
percentage_provincia2 = {key: round((value / sum(counter_provincia2.values())*100),2) for key,value in counter_provincia2.items()}
print(percentage_provincia2)

{'malaga': 12.72, 'cordoba': 21.3, 'jaen': 30.86, 'granada': 17.74, 'cadiz': 3.75, 'huelva': 1.17, 'almeria': 0.85, 'sevilla': 11.6}


4. Municipio

In [12]:
#Variable that identifies the town in Spain where the landplot is.
#This variable is, in principle, irrelevant, as there is more precise information regarding the specific location of the landplots.

print(feature_frame["municipio"].isnull().sum())
counter_municipio=Counter(feature_frame["municipio"])
print(counter_municipio)
print(len(counter_municipio))

0
Counter({'ubeda': 23870, 'loja': 17887, 'antequera': 16431, 'iznalloz': 16018, 'puente genil': 15084, 'alcala la real': 13999, 'baena': 13911, 'montefrio': 13228, 'baeza': 7928, 'colomera': 7926, 'adamuz': 7496, 'salar': 6392, 'alameda': 6358, 'mollina': 5776, 'algarinejo': 5580, 'illora': 5517, 'moclin': 5469, 'estepa': 5406, 'navas de san juan': 5293, 'martos': 5284, 'osuna': 5213, 'archidona': 5164, 'olvera': 5138, 'santaella': 5109, 'aguilar': 4928, 'campillos': 4836, 'rute': 4808, 'alcaudete': 4714, 'castro del rio': 4604, 'montillana': 4477, 'priego de cordoba': 4414, 'montejicar': 4346, 'santisteban del puerto': 4102, 'lucena': 4045, 'alhama de granada': 4043, 'pinar': 3958, 'torre del campo': 3891, 'luque': 3694, 'deifontes': 3543, 'castillo de locubin': 3448, 'cabra': 3442, 'torredonjimeno': 3343, 'setenil': 3263, 'cordoba': 3156, 'fuente de piedra': 3148, 'ecija': 3109, 'benalua de las villas': 3014, 'teba': 2936, 'nueva carteya': 2881, 'pozo alcon': 2816, 'bailen': 2759, '

In [13]:
percentage_municipio = {key: round((value / sum(counter_municipio.values())*100),2) for key,value in counter_municipio.items()}
print(percentage_municipio)

{'archidona': 0.96, 'antequera': 3.06, 'baena': 2.59, 'banos de la encina': 0.32, 'alcala la real': 2.61, 'martos': 0.99, 'torredonjimeno': 0.62, 'rus': 0.4, 'torre del campo': 0.73, 'ubeda': 4.45, 'navas de san juan': 0.99, 'montilla': 0.39, 'ibros': 0.31, 'canena': 0.5, 'santaella': 0.95, 'frailes': 0.49, 'victoria  la': 0.07, 'torreperogil': 0.25, 'castillo de locubin': 0.64, 'jaen': 0.28, 'linares': 0.41, 'porcuna': 0.24, 'arquillos': 0.23, 'montemayor': 0.03, 'castro del rio': 0.86, 'sabiote': 0.51, 'rambla  la': 0.42, 'fernan nunez': 0.04, 'bujalance': 0.42, 'nueva carteya': 0.54, 'cordoba': 0.59, 'luque': 0.69, 'espejo': 0.39, 'villanueva del arzobispo': 0.19, 'dona mencia': 0.05, 'canete de las torres': 0.45, 'fuerte del rey': 0.15, 'zuheros': 0.04, 'jabalquinto': 0.4, 'cabra': 0.64, 'cazalilla': 0.12, 'alcaudete': 0.88, 'obejo': 0.27, 'pozoblanco': 0.51, 'puerta de segura  la': 0.19, 'palma del rio': 0.5, 'cuevas del becerro': 0.04, 'almedinilla': 0.19, 'santisteban del puerto

5. Fecha

In [14]:
#This variable stores the date of the true measurement.
#The season happens of the olive happens mostly in winter, and most probably to put things more homogeneous we can
#delete 2003 and 2005 (and perhaps 2021), due to the lack of measurements with respect to the other years.
#This variable is, in principle, relevant.

print(feature_frame["fecha"].isnull().sum())
fecha_separated = pd.DataFrame(columns=['Year','Month','Day'])
fecha_separated['Day'] = feature_frame["fecha"].dt.day
fecha_separated['Month'] = feature_frame["fecha"].dt.month
fecha_separated['Year'] = feature_frame["fecha"].dt.year
print(fecha_separated.groupby(['Year','Month'])['Day'].count())

0
Year  Month
2005  8           6
      9          25
      10         27
      11         15
      12          6
2006  3        1502
      4        3098
      5        5240
      6        4542
      7        4789
      8        5179
      9        4930
      10       5490
      11       1218
2007  3        2379
      4        4261
      5        4820
      6        4639
      7        4887
      8        4646
      9        4865
      10       5667
      11       1992
2008  2           3
      3        2494
      4        4704
      5        4241
      6        4889
      7        4705
      8        4064
      9        5090
      10       4161
      11       1678
2009  3        4014
      4        5342
      5        5189
      6        6024
      7        5157
      8        5311
      9        5991
      10       4956
      11       2693
2010  3        3902
      4        5396
      5        5812
      6        5922
      7        4959
      8        5410
      9        5636
      

6. Campaña

In [15]:
#Campaña should be the same as year of measurement, easily obtainable by the previous or this variable.
#We see that there are slight differences between the fecha and campaña; however, they are generally less than 0.1%.
#It is, in principle, relevant.

print(feature_frame["campaña"].isnull().sum())
print(fecha_separated.groupby(['Year']).count()['Month'])

0
Year
2005       79
2006    35988
2007    38156
2008    36029
2009    44677
2010    43986
2011    42426
2012    29397
2013    29211
2014    29465
2015    31063
2016    28571
2017    36801
2018    35038
2019    37226
2020    36385
2021     1946
Name: Month, dtype: int64


In [16]:
for key, value in sorted(Counter(feature_frame["campaña"]).items()):
     print(key, value)

2005 74
2006 35940
2007 38163
2008 36068
2009 44673
2010 43962
2011 42410
2012 29426
2013 29233
2014 29458
2015 31070
2016 28571
2017 36731
2018 35111
2019 37223
2020 36385
2021 1946


In [17]:
percentage_fecha = {key: round((value / sum(Counter(feature_frame["campaña"]).values())*100),2) for key,value in Counter(feature_frame["campaña"]).items()}
print(percentage_fecha)

{2005: 0.01, 2006: 6.7, 2007: 7.11, 2008: 6.72, 2009: 8.33, 2010: 8.2, 2011: 7.91, 2012: 5.49, 2013: 5.45, 2014: 5.49, 2015: 5.79, 2016: 5.33, 2017: 6.85, 2018: 6.55, 2019: 6.94, 2020: 6.78, 2021: 0.36}


In [18]:
fechas_unique_codparcelas=feature_frame[["codparcela","campaña"]]
fechas_unique_codparcelas=fechas_unique_codparcelas.drop_duplicates()
print(fechas_unique_codparcelas.groupby(['campaña']).count())

         codparcela
campaña            
2005              6
2006           1402
2007           1343
2008           1276
2009           1442
2010           1430
2011           1461
2012           1021
2013           1053
2014           1033
2015           1185
2016           1116
2017           1442
2018           1418
2019           1528
2020           1444
2021            828


In [19]:
fechas_unique_codparcelas_first=feature_frame[["codparcela","campaña"]].sort_values('campaña')
fechas_unique_codparcelas_first=fechas_unique_codparcelas_first.drop_duplicates(subset="codparcela",keep="first")
print(fechas_unique_codparcelas_first.groupby(['campaña']).count())

         codparcela
campaña            
2005              6
2006           1396
2007            170
2008            232
2009            429
2010            334
2011            255
2012            174
2013            149
2014            165
2015            323
2016            207
2017            498
2018            142
2019            267
2020            185
2021            129


7. Poligono

In [20]:
#This value is the first of the 4 digit code that represents the landplot.
#There are less poligonos than municipios, which is not intuitive, and there are some null values.
#This variable is, in principle, relevant.

print(feature_frame["poligono"].isnull().sum())
counter_poligono=Counter(feature_frame["poligono"])
print(counter_poligono)
print(len(counter_poligono))

24
Counter({3: 23400, 4: 23201, 1: 23043, 2: 22902, 8: 21611, 7: 21547, 5: 21530, 9: 18706, 6: 18395, 10: 18018, 11: 16989, 15: 16082, 12: 15400, 13: 13736, 16: 12597, 17: 11428, 18: 10846, 25: 10476, 19: 10067, 14: 10046, 23: 8944, 20: 8909, 22: 8027, 21: 7662, 26: 6759, 27: 6338, 29: 6104, 43: 5613, 38: 5456, 35: 5394, 28: 5119, 24: 5112, 33: 4872, 32: 4775, 30: 4709, 42: 4581, 41: 4445, 37: 4357, 39: 4254, 36: 4097, 45: 3825, 31: 3810, 40: 3559, 34: 3021, 44: 2628, 51: 2509, 52: 2507, 49: 2404, 46: 2374, 55: 2186, 48: 2128, 47: 2074, 56: 1913, 62: 1787, 66: 1682, 87: 1639, 71: 1556, 50: 1537, 60: 1425, 70: 1331, 57: 1319, 53: 1258, 94: 1215, 83: 1192, 67: 1147, 58: 1090, 84: 1062, 59: 1001, 89: 999, 69: 979, 75: 974, 64: 966, 72: 889, 108: 853, 65: 835, 79: 794, 90: 782, 54: 739, 61: 738, 80: 727, 109: 688, 81: 669, 85: 656, 82: 600, 76: 575, 86: 546, 74: 515, 139: 487, 68: 457, 105: 433, 104: 426, 63: 406, 95: 379, 133: 368, 164: 363, 96: 337, 112: 324, 501: 305, 368: 296, 78: 266,

In [21]:
#We can rescue the null values as well as their codparcela.

feature_frame[feature_frame["poligono"].isnull()]


,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,days_until_next_visit,next_estado
47540,2020-04-23 14:34:36,R-879,cordoba,bujalance,2007-03-12,2007,<NA>,879,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2
47528,2020-04-23 14:34:36,R-875,cordoba,bujalance,2007-03-12,2007,<NA>,875,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,2
47513,2020-04-23 14:34:36,R-870,cordoba,bujalance,2007-03-13,2007,<NA>,870,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2
47534,2020-04-23 14:34:36,R-877,cordoba,bujalance,2007-03-13,2007,<NA>,877,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,2
47525,2020-04-23 14:34:36,R-874,cordoba,bujalance,2007-03-13,2007,<NA>,874,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2
47531,2020-04-23 14:34:36,R-876,cordoba,bujalance,2007-03-13,2007,<NA>,876,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,2
47537,2020-04-23 14:34:36,R-878,cordoba,bujalance,2007-03-13,2007,<NA>,878,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2
47522,2020-04-23 14:34:36,R-873,cordoba,bujalance,2007-03-14,2007,<NA>,873,<NA>,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,2
47516,2020-04-23 14:34:36,R-871,cordoba,bujalance,2007-03-14,2007,<NA>,871,<NA>,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

8. Parcela

In [22]:
#This value is the second of the 4 digit code that represents the landplot.
#There are some null values.
#This variable is, in principle, relevant.

print(feature_frame["parcela"].isnull().sum())
counter_parcela=Counter(feature_frame["parcela"])
print(counter_parcela)
print(len(counter_parcela))

29
Counter({1: 14060, 2: 11298, 3: 8928, 4: 8782, 7: 8007, 8: 7391, 6: 6948, 5: 6386, 16: 6380, 10: 6170, 14: 5876, 15: 5765, 21: 5536, 25: 4980, 12: 4979, 9: 4855, 13: 4722, 17: 4631, 20: 4611, 22: 4442, 30: 4357, 29: 4333, 49: 4010, 27: 3911, 24: 3829, 31: 3803, 33: 3803, 75: 3790, 18: 3755, 23: 3708, 67: 3671, 60: 3554, 41: 3454, 11: 3427, 28: 3369, 19: 3323, 55: 3242, 51: 3233, 54: 3171, 69: 3153, 44: 3062, 90: 3020, 35: 3002, 26: 2990, 48: 2945, 43: 2917, 64: 2887, 36: 2798, 32: 2797, 79: 2757, 66: 2690, 40: 2670, 42: 2655, 57: 2600, 166: 2600, 39: 2503, 45: 2432, 62: 2429, 59: 2426, 78: 2332, 73: 2150, 99: 2148, 72: 2125, 34: 2125, 65: 2113, 70: 2111, 37: 2085, 106: 2079, 96: 2072, 84: 2045, 88: 2044, 101: 2022, 76: 2012, 116: 1945, 145: 1944, 204: 1894, 38: 1878, 77: 1866, 109: 1861, 52: 1856, 61: 1850, 86: 1841, 56: 1839, 91: 1818, 104: 1810, 83: 1790, 47: 1772, 58: 1679, 46: 1665, 94: 1578, 100: 1578, 53: 1543, 176: 1541, 85: 1531, 141: 1513, 138: 1507, 214: 1497, 95: 1485, 68

In [23]:
#We can rescue the null values as well as their codparcela.

feature_frame[feature_frame["parcela"].isnull()]

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,days_until_next_visit,next_estado
121716,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-03-11,2009,21,<NA>,<NA>,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2
121717,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-03-18,2009,21,<NA>,<NA>,<NA>,0,2,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3
121718,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-03-25,2009,21,<NA>,<NA>,<NA>,0,1,2,0,0,0,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,3
121719,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-04-01,2009,21,<NA>,<NA>,<NA>,0,0,2,1,0,0,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,5
121720,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-04-15,2009,21,<NA>,<NA>,<NA>,0,0,0,1,2,0,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,5
121721,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-04-22,2009,21,<NA>,<NA>,<NA>,0,0,0,0,2,1,0,0,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,6
121722,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-04-29,2009,21,<NA>,<NA>,<NA>,0,0,0,0,1,2,1,0,0,0,0,0,0,0,1.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,6
121723,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-05-06,2009,21,<NA>,<NA>,<NA>,0,0,0,0,0,2,1,0,0,0,0,0,0,0,18.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,8
121724,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-05-13,2009,21,<NA>,<NA>,<NA>,0,0,0,0,0,0,1,2,0,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,9
121725,2020-04-23 14:34:36,021-002B,cordoba,santaella,2009-05-20,2009,21,<NA>,<NA>,<NA>,0,0,0,0,0,0,0,1,2,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN,N

9. Recinto

In [24]:
#This value is the third of the 4 digit code that represents the landplot.
#There are some null values.
#This variable is, in principle, relevant.

print(feature_frame["recinto"].isnull().sum())
counter_recinto=Counter(feature_frame["recinto"])
print(counter_recinto)
print(len(counter_recinto))

1601
Counter({0: 261571, 1: 144558, 2: 51852, 3: 23921, 4: 13939, 5: 8992, 6: 5416, 8: 4992, 7: 4275, 11: 2129, 9: 1716, <NA>: 1601, 14: 1502, 12: 1311, 16: 993, 10: 921, 13: 803, 15: 800, 17: 621, 20: 552, 31: 443, 36: 442, 34: 372, 25: 357, 24: 314, 18: 268, 21: 243, 26: 232, 44: 199, 27: 164, 29: 151, 40: 138, 37: 119, 32: 119, 39: 89, 56: 63, 46: 61, 19: 38, 90: 37, 64: 35, 28: 35, 63: 32, 49: 14, 51: 11, 23: 3})
45


In [25]:
percentage_recinto = {key: round((value / sum(counter_recinto.values())*100),2) for key,value in counter_recinto.items()}
print(percentage_recinto)

{0: 48.76, 1: 26.95, 5: 1.68, 4: 2.6, 2: 9.67, 6: 1.01, <NA>: 0.3, 3: 4.46, 7: 0.8, 8: 0.93, 24: 0.06, 9: 0.32, 64: 0.01, 10: 0.17, 14: 0.28, 90: 0.01, 11: 0.4, 27: 0.03, 46: 0.01, 12: 0.24, 13: 0.15, 25: 0.07, 31: 0.08, 16: 0.19, 51: 0.0, 15: 0.15, 44: 0.04, 63: 0.01, 20: 0.1, 36: 0.08, 21: 0.05, 39: 0.02, 17: 0.12, 49: 0.0, 34: 0.07, 18: 0.05, 37: 0.02, 26: 0.04, 40: 0.03, 32: 0.02, 19: 0.01, 29: 0.03, 56: 0.01, 28: 0.01, 23: 0.0}


In [26]:
#We can rescue the null values as well as their codparcela.

feature_frame[feature_frame["recinto"].isnull()]

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,days_until_next_visit,next_estado
47540,2020-04-23 14:34:36,R-879,cordoba,bujalance,2007-03-12,2007,<NA>,879,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2
47528,2020-04-23 14:34:36,R-875,cordoba,bujalance,2007-03-12,2007,<NA>,875,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,2
47513,2020-04-23 14:34:36,R-870,cordoba,bujalance,2007-03-13,2007,<NA>,870,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2
47534,2020-04-23 14:34:36,R-877,cordoba,bujalance,2007-03-13,2007,<NA>,877,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,2
47525,2020-04-23 14:34:36,R-874,cordoba,bujalance,2007-03-13,2007,<NA>,874,<NA>,<NA>,0,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,NaN,NaN,NaN,CAMPIÑA BAJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514328,2021-04-05 17:26:06,001-00006,jaen,ubeda,2020-09-01,2020,1,6,<NA>,<NA>,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2021-04-05 17:24:32,464989.0,4221345.0,NaN,LA LOMA,14.59,9.0,NaN,NaN,Secano,Si,Silvestre,NaN,10 X 10,80.0,3.0,NaN,NaN,"Picual, Marteño",NaN,NaN,NaN,NaN,NaN,JA/OL/01 SIERRA MORENA,JA001,NaN,NaN,VILCHES I,NaN,NaN,NaN,NaN,NaN,NaN,Producción Integrada (PI),8,10
514329,2021-04-05 17:26:06,001-00006,jaen,ubeda,2020-09-09,2020,1,6,<NA>,<NA>,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2021-04-05 17:24:32,464989.0,4221345.0,NaN,LA LOMA,14.59,9.0,NaN,NaN,Secano,Si,Silvestre,NaN,10 X 10,80.0,3.0,NaN,NaN,"Picual, Marteño",NaN,NaN,NaN,NaN,NaN,JA/OL/01 SIERRA MORENA,JA001,NaN,NaN,VILCHES I,NaN,NaN,NaN,NaN,NaN,NaN,Producción Integrada (PI),7,10
514330,2021-04-05 17:26:06,001-00006,jaen,ubeda,2020-09-16,2020,1,6,<NA>,<NA>,0,0,0,0,0,0,0,0,0,2,1,0,0,0

10. Subrecinto

In [27]:
#This value is the third of the 4 digit code that represents the landplot.
#There are some null values.
#This variable is, in principle, relevant.

print(feature_frame["subrecinto"].isnull().sum())
counter_subrecinto=Counter(feature_frame["subrecinto"])
print(counter_subrecinto)
print(len(counter_subrecinto))

96179
Counter({0: 315218, <NA>: 96179, 1: 91043, 2: 6163, 10: 5541, 30: 3144, 3: 2674, -1: 2644, 20: 2503, 4: 1383, 25: 1378, 5: 707, 6: 646, 40: 479, 60: 470, 8: 441, 11: 439, 23: 421, 7: 386, 80: 344, 18: 334, 50: 334, 26: 306, 14: 277, 15: 265, 45: 263, 94: 199, 70: 174, 9: 173, 61: 144, 12: 142, 44: 139, 32: 123, 93: 111, 92: 111, 55: 96, 81: 91, 98: 91, 17: 86, 75: 85, 42: 84, 33: 83, 62: 82, 22: 67, 90: 50, 52: 41, 63: 40, 16: 39, 56: 31, 91: 31, 97: 29, 13: 28, 24: 28, 46: 26, 38: 15, 19: 6, 31: 5, 99: 2, 67: 2, 21: 2, 85: 2, 73: 1, 95: 1, 64: 1, 34: 1})
65


In [28]:
percentage_subrecinto = {key: round((value / sum(counter_subrecinto.values())*100),2) for key,value in counter_subrecinto.items()}
print(percentage_subrecinto)

{0: 58.76, 60: 0.09, 1: 16.97, <NA>: 17.93, 23: 0.08, 20: 0.47, 10: 1.03, 56: 0.01, 80: 0.06, -1: 0.49, 40: 0.09, 70: 0.03, 3: 0.5, 6: 0.12, 14: 0.05, 4: 0.26, 8: 0.08, 7: 0.07, 2: 1.15, 30: 0.59, 90: 0.01, 99: 0.0, 18: 0.06, 13: 0.01, 16: 0.01, 50: 0.06, 11: 0.08, 25: 0.26, 73: 0.0, 5: 0.13, 81: 0.02, 46: 0.0, 31: 0.0, 52: 0.01, 15: 0.05, 24: 0.01, 44: 0.03, 97: 0.01, 45: 0.05, 42: 0.02, 17: 0.02, 62: 0.02, 94: 0.04, 75: 0.02, 33: 0.02, 55: 0.02, 98: 0.02, 9: 0.03, 12: 0.03, 91: 0.01, 93: 0.02, 92: 0.02, 26: 0.06, 63: 0.01, 61: 0.03, 32: 0.02, 22: 0.01, 67: 0.0, 21: 0.0, 38: 0.0, 85: 0.0, 19: 0.0, 95: 0.0, 64: 0.0, 34: 0.0}


In [29]:
#We can rescue the null values as well as their codparcela.
#Most of them are null because there is not a subrecinto in the parcela.

feature_frame[feature_frame["subrecinto"].isnull()]

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,days_until_next_visit,next_estado
399057,2020-04-23 17:00:15,025-00035-00-R0,malaga,canete la real,2006-03-21,2006,25,35,0,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,318933.0,4084189.0,700.0,NORTE O ANTEQUERA,4.000,14.0,S,FRANCO,SECANO,SI,SILVESTRE FORMADA CON HERBICIDA,NaN,11X11,83.0,3.0,NaN,NaN,HOJIBLANCO,01/03/1975,NaN,NaN,NaN,MA/OL/07,MA/OL/07 ANTEQUERA OCCIDENTAL,MA008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NINGUNO,NaN,NaN,7,2
289949,2020-04-23 16:54:05,012-00035-00-RA,jaen,ibros,2006-03-22,2006,12,35,0,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,451324.0,4212423.0,410.0,LA LOMA,8.924,5.0,N,NaN,REGADÍO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JA012,NaN,LOCALIZADO GOTERO,JA/OL/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2
3722,2020-04-23 14:31:50,013-00002-00-B0,cadiz,alcala del valle,2006-03-28,2006,13,2,0,<NA>,2,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,305729.0,4086549.0,670.0,SIERRA DE CADIZ - II,11.040,8.0,NaN,ARCILLOSO,SECANO,NO,NaN,NaN,10X10,104.0,3.0,NaN,NaN,"Picual, Marteño",NaN,NaN,NaN,LECHIN,CA/OL/05,CA/OL/05 SETENIL,CA003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2
12691,2020-04-23 14:31:50,001-00003-00-A0,cadiz,setenil,2006-03-28,2006,1,3,0,<NA>,2,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,304302.0,4088459.0,800.0,SIERRA DE CADIZ - II,15.360,7.0,NaN,ARCILLO-ARENOSO,SECANO,Si,Silvestre,NaN,8x8,118.0,3.0,NaN,NaN,"Picual, Marteño",NaN,NaN,NaN,NaN,CA/OL/05,CA/OL/05 SETENIL,CA003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1
399058,2020-04-23 17:00:15,025-00035-00-R0,malaga,canete la real,2006-03-28,2006,25,35,0,<NA>,1,2,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,318933.0,4084189.0,700.0,NORTE O ANTEQUERA,4.000,14.0,S,FRANCO,SECANO,SI,SILVESTRE FORMADA CON HERBICIDA,NaN,11X11,83.0,3.0,NaN,NaN,HOJIBLANCO,01/03/1975,NaN,NaN,NaN,MA/OL/07,MA/OL/07 ANTEQUERA OCCIDENTAL,MA008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NINGUNO,NaN,NaN,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537048,2021-04-05 17:26:06,010-00001-06,sevilla,alcala de guadaira,2021-03-23,2021,10,1,6,<NA>,0,0,2,1,0,0,0,0,0,0,0,0,0,0,NaN,2021-04-05 17:24:32,249206.0,4140344.0,78.0,LA CAMPIÑA,7.970,3.0,NaN,ARCILLO-ARENOSO,REGADÍO,No,NaN,NO,6X8,208.0,2.0,NaN,NaN,Manzanilla de Sevilla,2004,Pozo,Aptas para riego,Gordal Sevillana,NaN,SE/OL/09 LA VEGA,NaN,NaN,Localizado gotero,FINCA SOLEDAD,1980,Lechin de Sevilla,30.0,NaN,Ninguno,NaN,NaN,

26. generatedparcelas

In [30]:
#Variable that indicates the time when the parcela was introduced in the system/database.
#We have some null/not a time values, the rest of the parcelas have been inputted 2020-04-23 and 2021-04-05.
#This variable is, in principle, irrelevant.

print(feature_frame["generated_parcelas"].isnull().sum())
numvalues_generatedparcelas = len(set(feature_frame["generated_parcelas"]))
print(numvalues_generatedparcelas)
counter_generatedparcelas=Counter(feature_frame["generated_parcelas"])
print(counter_generatedparcelas)

3157
3
Counter({Timestamp('2020-04-23 14:19:57'): 385893, Timestamp('2021-04-05 17:24:32'): 147394, NaT: 3157})


27. 102_coordenada_x_(utm)

In [31]:
#Variable that indicates the x coordinate of the landplot. This is, pressumably, relevant.
#There are more null values than normal. The number of x_coordinates is aprox. the same as the num of codparcelas,
#so pressumably each codparcela has its own x_coordinate.
#In theory, there were 5239 codparcelas, and here there are 100 more x_coordinates, so some codparcelas
#may have different x_coordinates.

print(feature_frame["102_coordenada_x_(utm)"].isnull().sum())
numvalues_coordenada_x = len(set(feature_frame[feature_frame["102_coordenada_x_(utm)"].notnull()]["102_coordenada_x_(utm)"]))
print(numvalues_coordenada_x)
counter_coordenada_x=Counter(feature_frame["102_coordenada_x_(utm)"])
print(counter_coordenada_x)


9529
5140
Counter({339853.0: 693, 399430.0: 566, 379366.0: 518, 343214.0: 508, 328310.0: 498, 340370.0: 494, 470794.0: 494, 343399.0: 494, 338881.0: 489, 455886.0: 488, 341774.0: 484, 339123.0: 483, 369846.0: 482, 355972.0: 482, 345556.0: 479, 454682.0: 477, 373635.0: 477, 468732.0: 473, 347836.0: 464, 332146.0: 462, 383806.0: 462, 338634.0: 461, 297537.0: 458, 296763.0: 457, 314834.0: 456, 344418.0: 455, 289718.0: 455, 358548.0: 454, 352841.0: 450, 322629.0: 450, 298564.0: 450, 324849.0: 449, 253301.0: 448, 387552.0: 448, 308518.0: 447, 305532.0: 447, 357209.0: 445, 347840.0: 444, 340531.0: 442, 340739.0: 442, 356312.0: 440, 351751.0: 439, 347968.0: 432, 330599.0: 432, 306480.0: 431, 285242.0: 429, 349999.0: 424, 339820.0: 424, 370006.0: 424, 344517.0: 424, 362725.0: 423, 362745.0: 422, 317417.0: 419, 292050.0: 419, 350684.0: 418, 341128.0: 418, 341736.0: 417, 330839.0: 416, 287178.0: 415, 304180.0: 414, 338325.0: 413, 357730.0: 412, 368910.0: 411, 297153.0: 410, 294503.0: 410, 419205

In [32]:
#There are 498 landplots that have nulls in the altitude, and there are 5042 that do not have nulls in their altitudes.
#The sum of this is 5540, which is higher than the 5239 codparcelas that we have. Therefore, 301 codparcelas have both NA and a concrete number in
#their x_coordinate.

print(len(set(feature_frame[feature_frame["102_coordenada_x_(utm)"].isnull()]["codparcela"])))
print(len(set(feature_frame[feature_frame["102_coordenada_x_(utm)"].notnull()]["codparcela"])))

350
4938


28. 103_coordenada_y_(utm)

In [33]:
#Variable that indicates the y coordinate of the landplot. This is, pressumably, relevant.
#There are more null values than normal. The number of y_coordinates is aprox. the same as the num of codparcelas,
#so pressumably each codparcela has its own y_coordinate.
#In theory, there were 5239 codparcelas, and here there are 100 more y_coordinates, so some codparcelas
#may have different y_coordinates.

print(feature_frame["103_coordenada_y_(utm)"].isnull().sum())
numvalues_coordenada_y = len(set(feature_frame[feature_frame["103_coordenada_y_(utm)"].notnull()]["103_coordenada_y_(utm)"]))
print(numvalues_coordenada_y)
counter_coordenada_y=Counter(feature_frame["103_coordenada_y_(utm)"])
print(counter_coordenada_y)

9621
5098
Counter({4232831.0: 739, 4087315.0: 629, 4229668.0: 511, 4156052.0: 508, 4096787.0: 498, 4139733.0: 494, 4202116.0: 494, 4142124.0: 492, 4141168.0: 489, 4139756.0: 488, 4135978.0: 484, 4139646.0: 483, 4110714.0: 482, 4122336.0: 482, 4157133.0: 479, 4130286.0: 473, 4108554.0: 464, 4168516.0: 462, 4136127.0: 462, 4103551.0: 461, 4199544.0: 458, 4185124.0: 457, 4189965.0: 456, 4144782.0: 455, 4168245.0: 455, 4116063.0: 454, 4108414.0: 453, 4102826.0: 450, 4189654.0: 450, 4193139.0: 450, 4181266.0: 449, 4067720.0: 448, 4147294.0: 448, 4173592.0: 447, 4177112.0: 447, 4132606.0: 445, 4143795.0: 444, 4155996.0: 442, 4137207.0: 442, 4114385.0: 440, 4112918.0: 439, 4145572.0: 432, 4103208.0: 432, 4082520.0: 429, 4146012.0: 424, 4141178.0: 424, 4136580.0: 424, 4142251.0: 424, 4139534.0: 424, 4117041.0: 423, 4106819.0: 422, 4188630.0: 419, 4080039.0: 419, 4139046.0: 418, 4148534.0: 418, 4102650.0: 417, 4152607.0: 416, 4084260.0: 415, 4176566.0: 414, 4161221.0: 413, 4189061.0: 413, 40571

In [34]:
#There are 501 landplots that have nulls in the altitude, and there are 5041 that do not have nulls in their altitudes.
#The sum of this is 5542, which is higher than the 5239 codparcelas that we have. Therefore, 303 codparcelas have both NA and a concrete number in
#their y_coordinate.

print(len(set(feature_frame[feature_frame["103_coordenada_y_(utm)"].isnull()]["codparcela"])))
print(len(set(feature_frame[feature_frame["103_coordenada_y_(utm)"].notnull()]["codparcela"])))

354
4937


29. 104_altitud_(m)

In [35]:
#Variable that indicates the altitude coordinate of the landplot. This is, pressumably, relevant.
#There are many many more null values than normal.
#There are much less non null values than codparcelas, so most probably most landplots won't have an altitude value.

print(feature_frame["104_altitud_(m)"].isnull().sum())
numvalues_altitud = len(set(feature_frame[feature_frame["104_altitud_(m)"].notnull()]["104_altitud_(m)"]))
print(numvalues_altitud)
counter_altitud=Counter(feature_frame["104_altitud_(m)"])
print(counter_altitud)

176080
571
Counter({500.0: 13938, 600.0: 11862, 700.0: 11654, 400.0: 11181, 450.0: 8516, 900.0: 8122, 800.0: 7436, 650.0: 6906, 300.0: 6583, 750.0: 6233, 550.0: 5843, 480.0: 5303, 350.0: 4956, 420.0: 4942, 320.0: 4832, 850.0: 4508, 560.0: 4249, 580.0: 4242, 950.0: 4145, 680.0: 3786, 640.0: 3765, 200.0: 3760, 250.0: 3565, 620.0: 3520, 520.0: 3441, 460.0: 3402, 360.0: 3097, 1000.0: 2998, 720.0: 2909, 380.0: 2901, 430.0: 2887, 440.0: 2724, 280.0: 2703, 690.0: 2696, 390.0: 2638, 540.0: 2294, 780.0: 2294, 820.0: 2199, 980.0: 2128, 410.0: 2101, 730.0: 2039, 490.0: 1903, 1050.0: 1826, 1100.0: 1775, 510.0: 1668, 425.0: 1664, 610.0: 1645, 530.0: 1611, 340.0: 1571, 310.0: 1539, 590.0: 1518, 630.0: 1431, 220.0: 1381, 270.0: 1372, 990.0: 1354, 330.0: 1320, 890.0: 1296, 805.0: 1287, 570.0: 1269, 760.0: 1244, 660.0: 1237, 790.0: 1234, 670.0: 1218, 470.0: 1218, 930.0: 1218, 840.0: 1191, 1180.0: 1160, 625.0: 1148, 230.0: 1148, 740.0: 1143, 260.0: 1135, 1060.0: 1116, 755.0: 1094, 290.0: 1086, 240.0: 10

In [36]:
#There are 2616 landplots that have nulls in the altitude, and there are 3047 that do not have nulls in their altitudes.
#The sum of this is 5663, which is higher than the 5239 codparcelas that we have. Therefore, 424 codparcelas have both NA and a concrete number in
#their altitudes.

print(len(set(feature_frame[feature_frame["104_altitud_(m)"].isnull()]["codparcela"])))
print(len(set(feature_frame[feature_frame["104_altitud_(m)"].notnull()]["codparcela"])))

2430
3025


30. 105_comarca

In [37]:
#Variable that indicates one subregion variable.
#Most probably, it will only be relevant the subregion, or the region, or the town... but not all of them at once.

print(feature_frame["105_comarca"].isnull().sum())
counter_comarca=Counter(feature_frame["105_comarca"])
print(counter_comarca)

3558
Counter({'NORTE O ANTEQUERA': 56933, 'LA LOMA': 47967, 'IZNALLOZ': 44354, 'DE LA  VEGA': 35671, 'CAMPIÑA ALTA': 31152, 'CAMPIÑA BAJA': 30713, 'MONTEFRIO': 29762, 'CAMPIÑA DEL SUR': 20805, 'PENIBETICA': 18989, 'ESTEPA': 15536, 'CAMPIÑA ALTA - II': 14765, 'SIERRA SUR - II': 13999, 'SIERRA SUR': 13990, 'LA CAMPIÑA': 13403, 'EL CONDADO': 12762, 'CAMPIÑA DEL NORTE - II': 9473, 'MAGINA': 8635, 'SIERRA DE CADIZ': 8570, 'ALHAMA': 8400, 'LA SIERRA SUR': 7719, 'LA SIERRA - II': 7496, 'SIERRA DE CADIZ - II': 6563, 'LA SIERRA': 6451, 'SIERRA DE SEGURA': 6135, 'PEDROCHES': 6127, 'SIERRA DE CAZORLA': 5776, 'LA SIERRA - III': 5044, nan: 3558, 'HUESCAR': 3453, 'LA SIERRA SUR - II': 3428, 'CONDADO CAMPIÑA': 3385, 'LAS COLONIAS': 3049, 'SERRANIA DE RONDA - II': 2597, 'CAMPIÑA DE CADIZ': 2578, 'CONDADO LITORAL': 2003, 'CAMPIÑA ALTA - III': 1994, 'SIERRA MORENA': 1934, 'MAGINA - II': 1894, 'CAMPIÑA DEL NORTE': 1829, 'COSTA - II': 1778, 'SIERRA DE CAZORLA - II': 1678, 'VALLE DE LECRIN': 1511, 'CAMPO T

In [38]:
percentage_comarca = {key: round((value / sum(counter_comarca.values())*100),2) for key,value in counter_comarca.items()}
print(percentage_comarca)

{'NORTE O ANTEQUERA': 10.61, 'CAMPIÑA ALTA': 5.81, 'SIERRA MORENA': 0.36, 'SIERRA SUR - II': 2.61, 'CAMPIÑA DEL SUR': 3.88, 'LA LOMA': 8.94, 'EL CONDADO': 2.38, 'CAMPIÑA ALTA - III': 0.37, 'CAMPIÑA BAJA': 5.73, 'SIERRA SUR': 2.61, 'LAS COLONIAS': 0.57, 'CAMPIÑA DEL NORTE - II': 1.77, 'CAMPIÑA DEL NORTE': 0.34, 'PENIBETICA': 3.54, 'LA SIERRA': 1.2, 'PEDROCHES': 1.14, 'SIERRA DE SEGURA': 1.14, 'SERRANIA DE RONDA': 0.17, 'SERRANIA DE RONDA - II': 0.48, 'MAGINA - II': 0.35, 'MAGINA': 1.61, 'LA SIERRA - II': 1.4, 'MONTEFRIO': 5.55, 'LA SIERRA - III': 0.94, 'SIERRA DE CAZORLA': 1.08, 'SIERRA DE CADIZ': 1.6, 'CAMPIÑA DE CADIZ': 0.48, 'CAMPIÑA DEL NORTE - III': 0.02, 'CAMPIÑA ALTA - II': 2.75, 'CENTRO SUR O GUADALHORCE': 0.15, 'DE LA  VEGA': 6.65, 'VELEZ-MALAGA': 0.06, 'CONDADO LITORAL': 0.37, 'BAZA': 0.26, 'SIERRA DE CADIZ - II': 1.22, 'LAS ALPUJARRAS': 0.08, 'VALLE DE LECRIN': 0.28, 'ALHAMA': 1.57, 'ALTO ALMANZORA': 0.04, 'BAJO ALMANZORA': 0.04, 'IZNALLOZ': 8.27, 'CONDADO CAMPIÑA': 0.63, 'CA

31. 201_superf_cultivada_en_la_parcela_agrícola_(ha)

In [39]:
#Variable that indicates the surface of cultivated land in ha.
#Most probably this is a relevant variable; however, this statistics are not representative, as some landplots have more recorded measurements
#than others.

print(feature_frame["201_superf_cultivada_en_la_parcela_agrícola_(ha)"].isnull().sum())
feature_frame["201_superf_cultivada_en_la_parcela_agrícola_(ha)"].describe()

46427


count    490017.000000
mean         53.837608
std         302.184509
min           0.000000
25%           2.000000
50%           6.130000
75%          21.430000
max       35781.000000
Name: 201_superf_cultivada_en_la_parcela_agrícola_(ha), dtype: float64

In [40]:
#To get more relevant statistics, we should delete the null values from the analysis and groupby landplot code.

surface_notnull=feature_frame[feature_frame["201_superf_cultivada_en_la_parcela_agrícola_(ha)"].notnull()]
surface_notnull_grouped=surface_notnull.groupby("codparcela").mean()["201_superf_cultivada_en_la_parcela_agrícola_(ha)"]
surface_notnull_grouped

<ipython-input-40-83e1ec4458c0>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  surface_notnull_grouped=surface_notnull.groupby("codparcela").mean()["201_superf_cultivada_en_la_parcela_agrícola_(ha)"]


codparcela
000-00000-00         NaN
000-00000-00-0     14.10
000-00900-00-U0     2.65
000-00999-00-00     0.80
000-00999-00-FC     0.80
                   ...  
R-879                NaN
RAIF-2               NaN
RAIF-3               NaN
RAIF-4               NaN
RAIF-5               NaN
Name: 201_superf_cultivada_en_la_parcela_agrícola_(ha), Length: 5239, dtype: float32

In [41]:
#The fact that the maximum while aggregating and averaging the cultivated surface is not the same as the previous case
#means that there are some landplots where the surface of cultivated land changes with time.

surface_notnull_grouped.describe()

count    4698.000000
mean       44.868813
std       190.855972
min         0.000000
25%         2.000000
50%         5.930500
75%        20.070499
max      7025.252441
Name: 201_superf_cultivada_en_la_parcela_agrícola_(ha), dtype: float64

32. 202_pendiente_(%)

In [42]:
#Variable that indicates the slope of the land
#Majority of the dataset includes a null value. The majority of the values are below 10%.
#This variable may be relevant for the model.

print(feature_frame["202_pendiente_(%)"].isnull().sum())
feature_frame["202_pendiente_(%)"].describe()

185343


count    351101.000000
mean         10.204628
std           8.584397
min           0.000000
25%           4.000000
50%           8.000000
75%          15.000000
max          64.000000
Name: 202_pendiente_(%), dtype: float64

33. 203_orientación

In [43]:
#It is not clear what orientation means: axis of measurement, where does the wind come from...
#It requires homogenizing, as some components are repeated using different names: "SUR ESTE" and "SURESTE", for instance.
#Not clear if this is relevant

print(feature_frame["203_orientación"].isnull().sum())
counter_orientacion=Counter(feature_frame["203_orientación"])
print(counter_orientacion)

310032
Counter({nan: 310032, 'N': 21286, 'S': 21157, '2 - NE': 18587, '1 - N': 16104, '4 - SE': 15661, '5 - S': 15184, 'SE': 14311, 'NE': 11592, '3 - E': 10323, 'NO': 10266, '0 - Llana': 9498, '8 - NO': 9315, 'E': 9042, '6 - SO': 8836, '7 - O': 7976, '9 - Varias': 6374, 'O': 5964, 'SO': 5526, 'LLANO': 2570, 'N-S': 1393, 'X': 1327, 'EO': 714, '1/2 N 1/2 S': 685, 'LL': 577, 'LLANA': 488, 'ESTE': 362, 'NS': 276, 'E-NE': 170, 'E-O': 160, 'SIN DEFINIR': 139, 'SUROESTE': 136, 'N-E': 128, 'llano': 76, 'SUR ESTE': 71, 'SURESTE': 71, 'N-O': 67})


34. 204_textura_del_suelo

In [44]:
#It is the type of soil the landplot is in
#It requires homogenizing, as some components are repeated using different names: "FRANCO-ARCILLOSO", "Franco-arcilloso", "Franco arcilloso", for instance.
#Not clear if this is relevant

print(feature_frame["204_textura_del_suelo"].isnull().sum())
counter_textura=Counter(feature_frame["204_textura_del_suelo"])
print(counter_textura)

292643
Counter({nan: 292643, 'FRANCO-ARCILLOSO': 47827, 'ARCILLO-ARENOSO': 24502, 'FRANCO': 22862, 'FRANCO-ARENOSO': 20884, 'ARCILLOSO': 19198, 'Franco-arcilloso': 16169, 'FRANCO-ARCILLO-ARENOSO': 12328, 'Calizas y margas (localmente areniscas o radiolari': 9077, 'ARCILLO-LIMOSO': 8990, 'FRANCO-ARCILLO-LIMOSO': 7026, 'ARENO-FRANCO': 6575, 'Franco': 4931, 'Arcilloso': 4776, 'Areno-franco': 4754, 'Conglomerados, arenas, lutitas y calizas': 4305, 'Margas, margocalizas, calizas (localmente calcaren': 4170, 'ARENOSO': 3856, 'Franco-arcillo-arenoso': 3310, 'FRANCO-LIMOSO': 2690, 'Calcarenitas, arenas, margas y calizas': 2348, 'Franco-arenoso': 2279, 'Margas yesíferas, areniscas y calizas': 1939, 'MEDIA': 1187, 'Margas, areniscas y lutitas o silexitas': 1125, 'Franco-arcillo-limoso': 1004, 'Arcillo-arenoso': 796, 'FRANCO ARCILLOSO': 779, 'Arenas, limos, arcillas, gravas y cantos': 647, 'Limoso': 561, 'Arenoso': 373, 'FRANCO CON ELEMENTOS GRUESOS': 372, 'Franco-Arenoso-Arcilloso': 362, 'Margas

35. 206_secano_/_regadío

In [45]:
#It defines if the landplot is watered or not: https://www.gestiriego.com/diferencia-entre-agricultura-de-secano-y-regadio/
#Tentatively, there should only be 3 categories: Secano, Regadio, Riego de apoyo.
#It requires homogenizing, as some components are repeated using different names: "REGADÍO", "Regadío", "REGADIO", for instance.
#Most probably this is relevant.

print(feature_frame["206_secano_/_regadío"].isnull().sum())
counter_secano_regadio=Counter(feature_frame["206_secano_/_regadío"])
print(counter_secano_regadio)

62789
Counter({'SECANO': 212267, 'REGADÍO': 123231, 'Secano': 70679, nan: 62789, 'Regadío': 41731, 'RIEGO DE APOYO': 18403, 'Riego de apoyo': 4397, 'REGADIO': 2107, 'SECANO CON RIEGO DE APOYO': 348, 'RIEGO': 337, 'RlEGO': 116, 'Regadio': 39})


In [46]:
percentage_secano_regadio = {key: round((value / sum(counter_secano_regadio.values())*100),2) for key,value in counter_secano_regadio.items()}
print(percentage_secano_regadio)

{'SECANO': 39.57, 'RIEGO DE APOYO': 3.43, 'REGADÍO': 22.97, nan: 11.7, 'REGADIO': 0.39, 'Regadío': 7.78, 'Secano': 13.18, 'SECANO CON RIEGO DE APOYO': 0.06, 'RIEGO': 0.06, 'RlEGO': 0.02, 'Regadio': 0.01, 'Riego de apoyo': 0.82}


36. 211_utilización_de_cubierta_vegetal

In [47]:
#It defines if the landplot is left with some grass growing, or not: https://gesmontes.es/olivo-y-cubierta-vegetal/
#Tentatively, there should only be or 2 definite categories, or some kind of spectrum (here 2 categories only, YES/NO)
#It requires homogenizing, as some components are repeated using different names: "Si", "SI", for instance.
#Most probably this is relevant.

print(feature_frame["211_utilización_de_cubierta_vegetal"].isnull().sum())
counter_cubiertavegetal=Counter(feature_frame["211_utilización_de_cubierta_vegetal"])
print(counter_cubiertavegetal)

148666
Counter({'Si': 182532, nan: 148666, 'NO': 81850, 'No': 72416, 'SI': 50980})


In [48]:
percentage_cubiertavegetal = {key: round((value / sum(counter_cubiertavegetal.values())*100),2) for key,value in counter_cubiertavegetal.items()}
print(percentage_cubiertavegetal)

{'SI': 9.5, 'NO': 15.26, nan: 27.71, 'No': 13.5, 'Si': 34.03}


37. 212_tipo_de_cubierta_vegetal

In [49]:
#It defines what kind of grass / vegetation is left: https://gesmontes.es/olivo-y-cubierta-vegetal/
#Tentatively, there should only be or 2 definite categories, or some kind of spectrum (here 2 categories only, YES/NO)
#It requires homogenizing, as some components are repeated using different names: "Si", "SI", for instance.
#Most probably this is relevant.

print(feature_frame["212_tipo_de_cubierta_vegetal"].isnull().sum())
counter_tipocubiertavegetal=Counter(feature_frame[feature_frame["212_tipo_de_cubierta_vegetal"].notnull()]["212_tipo_de_cubierta_vegetal"])
print(counter_tipocubiertavegetal)

301766
Counter({'Silvestre': 150457, 'SILVESTRE FORMADA CON HERBICIDA': 38488, 'ESPONTANEA': 13835, 'Inerte ( restos poda, ..)': 12449, 'SILVESTRE FORMADA CON LABORES': 7810, 'Instalada con siembra': 2613, 'INSTALADA CON SIEMBRA': 2155, 'INERTE (RESTOS DE PODA)': 1304, 'NO': 1215, 'NATURAL': 1108, 'NINGUNA': 853, 'SILVESTRE': 365, 'CONTROLADA CON GANADO': 342, 'RESTOS VEGETALES': 322, 'SILVESTRE DESBROZADORA (PARCELAS)': 259, 'SILVESTRE FORMADA CON HERBICIDA Y RESTOS DE PODA': 210, 'PARCIAL': 161, 'PIEDRA NATURAL': 158, 'SIN DEFINIR': 139, 'SI': 71, 'EXPONTANEA': 71, 'INSTALADA CON HERBICIDAS': 71, 'RESTOS PODA': 56, 'CUBIERTA SILVESTRE CON DESBROZADORA': 53, 'SILVESTRE CONTROLADA POR PASTOREO': 39, 'SE INSTALARA EN SEPTIENBRE 2006': 27, 'SILVESTRE FORMADA CON GANADO OVINO': 24, 'FLORA NATURAL': 23})


In [50]:
#The sums of the vegetation type approximately sum the variable 36) if there is some vegetation, so there is only a slight mismatch there.

sum(counter_tipocubiertavegetal.values())

234678

38. 214_cultivo_asociado/otro_aprovechamiento

In [51]:
#It defines if there is another crop being grown with the olive trees
#It requires homogenizing, as some components are repeated using different names: "NO", "No", "Ninguno", "NINGUNO" for instance.
#Most probably this is relevant.

print(feature_frame["214_cultivo_asociado/otro_aprovechamiento"].isnull().sum())
counter_cultivoasociado=Counter(feature_frame["214_cultivo_asociado/otro_aprovechamiento"])
print(counter_cultivoasociado)

478890
Counter({nan: 478890, 'NO': 50837, 'No': 2657, 'NINGUNO': 2510, 'Ninguno': 349, 'Almendro': 256, 'CUBIERTA ESPONTANEA': 255, 'CEREZO': 189, 'CITRICOS EN ALGUNA DE LAS CALLES': 169, 'SI': 130, 'ALGUNOS FRUTALES COMO MANZANOS Y CEREZOS': 66, 'CEREZOS EN PARTE DE LA PARCELA': 60, 'CEREAL': 43, 'CITRICOS': 33})


39. 301_marco_(m_x_m)

In [52]:
#It defines the separation of the olive trees with respect to each other: https://espacioagroecologicodelasubbetica.blogspot.com/2012/08/marco-de-plantacion-y-tipo-de-hoyo.html, https://www.viveroscriado.es/plantaciones-de-olivos/
#It requires homogenizing, as some components are repeated using different names: "10 X 10", "10x10", "10*10", for instance.
#Most probably this is relevant. Very related to other parameters e.g. regadio fields have smaller marco than secano fields

print(feature_frame["301_marco_(m_x_m)"].isnull().sum())
counter_marco=Counter(feature_frame["301_marco_(m_x_m)"])
print(counter_marco)

75821
Counter({'10 X 10': 98053, nan: 75821, '8 X 8': 38268, '9 X 9': 34385, '7 X 7': 21781, '12 X 12': 20080, '10 x 10': 9807, '7X7': 9610, '10x10': 9568, '10*10': 9184, '7x7': 9073, '10X10': 7457, '11 X 11': 7020, '11x11': 6784, '11X11': 6033, '7 x 7': 5806, '8x8': 5521, '12x12': 3939, '7*7': 3678, '12X12': 3660, '12*12': 3646, '10  X 10': 3539, '9X9': 3227, '11*11': 3200, '14 X 14': 2863, '12 x 12': 2660, '8 X 9': 2516, '9 x 9': 2396, '7 X 8': 2167, '7 X 5': 2167, '13 X 13': 2116, '8.4*8.4': 2098, '8.9*8.9': 1808, '8.3*8.3': 1729, '6 X 8': 1701, '8 x 8': 1514, '9 X 10': 1453, 'MARCO IRREGULAR': 1445, '8X7': 1386, '8*8': 1379, '9x9': 1346, '12x12 tresbolillo': 1328, '6 X 6': 1315, '6 X 7': 1315, '7x6': 1313, '4 X 1.5': 1288, '7 X 6': 1251, '10*10 REAL': 1244, '8 X 4': 1237, '9*9': 1200, '8 x 6': 1114, '10 X 12': 1111, '7*5': 1099, '10 x 12': 1081, '7 x 6': 1063, '10 X 11': 1059, '12X10': 1029, '6 X 12': 1029, '8.6*8.6': 1016, '8.7*8.7': 997, '12 x12': 980, '8 x 4': 980, '8X8': 976, '

40. 302_densidad_(plantas/ha)

In [53]:
#It defines the density of olive trees, like plants/hectar. 1 hectar = 10000 m2
#This should be redundant or deeply related to the marco:
#having a marco of 10 x 10 m means that in 1 hectar we have 100 olive trees
#having a marco of 9 x 9 m means that in 1 hectar we have 123 olive trees
#having a marco of 8 x 8 m means that in 1 hectar we have 156 olive trees
#having a marco of 7 x 7 m means that in 1 hectar we have 204 olive trees
#Suspiciously, these numbers match with the ones in the Counter, so maybe it is best just to calculate this ratio rather than get it from this column with nulls.

print(feature_frame["302_densidad_(plantas/ha)"].isnull().sum())
counter_densidad=Counter(feature_frame[feature_frame["302_densidad_(plantas/ha)"].notnull()]["302_densidad_(plantas/ha)"])
print(counter_densidad)

73764
Counter({100.0: 126310, 204.0: 28083, 156.0: 20726, 70.0: 16043, 200.0: 15606, 123.0: 12667, 120.0: 10016, 90.0: 9812, 80.0: 9329, 130.0: 8198, 69.0: 8104, 83.0: 7552, 150.0: 7265, 238.0: 6310, 82.0: 5124, 110.0: 5121, 208.0: 5068, 85.0: 4673, 125.0: 4224, 138.0: 3915, 285.0: 3504, 178.0: 3361, 140.0: 3148, 250.0: 2640, 60.0: 2615, 95.0: 2607, 155.0: 2565, 160.0: 2483, 144.0: 2456, 312.0: 2134, 115.0: 2104, 139.0: 2078, 157.0: 2045, 205.0: 1946, 105.0: 1877, 286.0: 1822, 51.0: 1816, 98.0: 1742, 127.0: 1673, 154.0: 1590, 124.0: 1541, 84.0: 1509, 122.0: 1498, 128.0: 1386, 1666.0: 1368, 145.0: 1297, 101.0: 1283, 116.0: 1263, 65.0: 1252, 74.0: 1243, 170.0: 1232, 210.0: 1216, 133.0: 1202, 179.0: 1189, 75.0: 1182, 121.0: 1166, 137.0: 1160, 163.0: 1153, 86.0: 1116, 143.0: 1111, 126.0: 1106, 76.0: 1100, 81.0: 1099, 151.0: 1077, 103.0: 1076, 88.0: 1064, 408.0: 1044, 175.0: 1044, 206.0: 1039, 113.0: 1030, 107.0: 1027, 94.0: 997, 102.0: 995, 142.0: 986, 114.0: 984, 240.0: 974, 131.0: 957, 6

41. 303_nº_pies_por_árbol

In [54]:
#It defines the number of trunks that come from the same olive tree, approximately. https://www.monteoliva.com/formacion/reconversion-del-olivar-tradicional-38
#It may be a false friend because in some places it is defined as the density (plants / ha) too.
#It may be important

print(feature_frame["303_nº_pies_por_árbol"].isnull().sum())
counter_pies=Counter(feature_frame[feature_frame["303_nº_pies_por_árbol"].notnull()]["303_nº_pies_por_árbol"])
print(counter_pies)

127049
Counter({3.0: 151233, 2.0: 124966, 1.0: 115695, 4.0: 16610, 123.0: 302, 12.0: 221, 69.0: 129, 0.0: 100, 193.0: 82, 5.0: 57})


42. 304_formación

In [55]:
#This can define how the tree is formed and perhaps cutted. https://www.youtube.com/watch?v=oKGj9f_URJ0
#It requires homogenizing, as some components are repeated using different names: "Vaso", "vaso", for instance.
#It may be important

print(feature_frame["304_formación"].isnull().sum())
counter_formacion=Counter(feature_frame["304_formación"])
print(counter_formacion)

391156
Counter({nan: 391156, 'Vaso': 78191, 'VASO': 31953, 'MARCO REAL': 5302, 'REAL': 3424, 'TRADICIONAL': 2517, 'vaso': 1940, 'REJUVENECIMIENTO': 1924, 'Espaldera': 1673, 'TRESBOLILLO': 1515, 'VAS0': 1488, 'Natural': 1422, 'FORMACION': 1274, 'PODA DE RENOVACIÓN': 1271, 'EN VASO LIBRE': 1088, 'NATURAL': 985, 'PRODUCCION Y REJUVENECIMIENTO': 907, 'MARCO TRESBOLILLO': 905, 'LIBRE': 700, 'cuadro': 541, '3 PIES': 531, 'De Producción': 409, 'PODA DE FORMACIÓN': 355, 'tresbolillo': 350, 'ENBASO': 320, 'TRADICIONAL VARIOS PIES': 319, 'PODA JAEN': 314, '2 PIES': 301, 'FORMACION Y PRODUCCION': 292, 'FORMACION Y REJUVENECIMIENTO': 285, 'VARIABLE': 278, 'VARIOS PIES': 207, '3 pies': 181, 'PALANCAS 1 PIE': 176, 'PLANTACIÓN IRREGULAR': 158, 'CINCO DE OROS': 152, 'ALEATORIA': 150, '10 METROS': 147, 'rectangulo': 145, 'SIN DEFINIR': 139, 'PODA DE RENOVACION': 138, 'Rejuvenecimiento': 127, 'TRADICIONAL 3-4 PIES': 122, 'GLOBO': 121, 'UN PIE': 112, 'INTENSIVO': 65, 'De formación': 61, 'Tradicional': 58

43. 305_diámetro_de_copa_(m)

In [56]:
#This defines the size of the tree crown.
#It has to be checked, because it is impossible an olive tree with a tree crown of 10000 m (maximum)
#It may be important

print(feature_frame["305_diámetro_de_copa_(m)"].isnull().sum())
feature_frame["305_diámetro_de_copa_(m)"].describe()

422610


count    113834.000000
mean         47.859398
std         652.414673
min           0.000000
25%           4.000000
50%           4.000000
75%           5.500000
max       10000.000000
Name: 305_diámetro_de_copa_(m), dtype: float64

44. 308_variedad_principal

In [57]:
#This defines the variety of olive tree / field. https://www.youtube.com/watch?v=oKGj9f_URJ0
#It requires homogenizing, as some components are repeated using different names: "Picual, Marteño", "PICUAL / MARTEÑO", "PICUAL", for instance.
#This might be a very relevant variable.

print(feature_frame["308_variedad_principal"].isnull().sum())
counter_variedad=Counter(feature_frame["308_variedad_principal"])
print(counter_variedad)

51969
Counter({'Picual, Marteño': 169989, 'Hojiblanca, Lucentino': 86221, 'PICUAL / MARTEÑO': 84425, nan: 51969, 'HOJIBLANCO': 39318, 'PICUAL': 29206, 'Manzanilla de Sevilla': 10853, 'Arbequina': 9319, 'Lechín de Sevilla, Zorzaleño, Ecijano': 6582, 'Picudo': 5987, 'MANZANILLO': 5082, 'PICUDO': 4985, 'Nevadillo Negro': 4552, 'LECHIN': 3601, 'HOJIBLANCA': 3141, 'NEVADILLO BLANCO': 2971, 'Picual': 2742, 'ARBEQUINA': 1997, 'Gordal Sevillana': 1959, 'VERDIAL': 1688, 'Cornicabra': 734, 'Verdial de Huévar': 669, 'NEVADILLO': 668, 'PICUAL Y PICUDO': 636, 'Lucio': 633, 'Verdial de Vélez-Málaga': 626, 'ALOREÑA': 593, 'PICUDO/PICUAL': 505, 'PICUAL/MARTEÑO': 478, 'Aceite+Mesa': 457, 'Lechín de Sevilla': 452, 'MARTEÑO': 429, 'LUCIO': 345, 'MARTEÑA': 291, 'ARBEQUINA (15000 OLIVOS)': 267, 'MANZ. ZAHURD 87-C.ALIZN 72-COTO 86- DETR. NAVE 98': 266, 'Royal de Cazorla': 161, 'PICUDO Y MARTEÑO': 160, 'Hojiblanca': 152, 'Manzanilla Cacereña': 146, 'Lechín de Granada, Cuquillo': 145, 'PICUDO/HOJIBLANCO': 140

45. 311_fecha_de_plantación_variedad_principal

In [58]:
#This field records the age of the olive trees planted.
#In most cases, this age is not registered, so depending on the quality of the rest of the data this can be deleted.

print(feature_frame["311_fecha_de_plantación_variedad_principal"].isnull().sum())
counter_variedad=Counter(feature_frame["311_fecha_de_plantación_variedad_principal"])
print(counter_variedad)

402952
Counter({nan: 402952, '01/03/1950': 7523, '01/03/1990': 5750, '01/03/1960': 5196, '01/03/1900': 5150, '01/03/1995': 4938, '1950': 3615, '01/03/1970': 3120, '01/03/1980': 2803, '01/03/1940': 1881, '1900': 1819, '01/03/1996': 1791, '01/03/1975': 1569, '01/03/1993': 1532, '01/03/2000': 1468, '01/03/1955': 1424, '01/03/1998': 1423, '01/03/1930': 1399, '01/03/1920': 1371, '25/05/1900': 1254, '01/03/1903': 1252, '01/01/1920': 1213, '1940': 1088, '01/03/1982': 1063, '01/01/1900': 1046, '01/11/1900': 1004, '01/03/1985': 983, '1999': 963, '01/03/1992': 963, '01/03/1991': 944, '1990': 918, '01/11/1950': 779, '01/05/1910': 775, '01/01/1998': 770, '17/03/1900': 767, '01/01/1970': 741, '01/03/1994': 740, '1987': 712, '01/03/1965': 701, '01/03/1958': 700, '01/03/1986': 680, '1995': 673, '01/01/1950': 661, '1997': 638, '1850': 617, '01/03/2005': 615, '01/03/1935': 607, '01/01/1993': 598, '2004': 597, '01/03/1906': 582, '1960': 578, '01/10/1990': 568, '20/03/1990': 549, '1992': 548, '1993': 547

46. 208_riego:_procedencia_del_agua

In [59]:
#This field records where the water comes from in each case
#This mixes specific locations ("EMBALSE LA BOLERA", "C.R. los Barrancos") with non specific things at all "Pozo"
#In most cases, this is not registered, so depending on the quality of the rest of the data this can be deleted.

print(feature_frame["208_riego:_procedencia_del_agua"].isnull().sum())
counter_procedagua=Counter(feature_frame["208_riego:_procedencia_del_agua"])
print(counter_procedagua)

408955
Counter({nan: 408955, 'Pozo': 37888, 'POZO': 32203, 'Balsa': 8724, 'BALSA': 8610, 'RÍO': 8013, 'Rio': 7658, 'Canal': 5826, 'CANAL': 4493, 'MANANTIAL': 2239, 'NACIMIENTO': 2188, 'Comunidad Regantes': 905, 'RESIDUAL': 763, 'Balsa (Comunidad de Regantes)': 707, '-': 701, 'EMBALSE LA BOLERA': 685, 'COMUNIDAD DE REGANTES': 640, 'Acequia': 499, 'Comunidad de Regantes': 445, 'Pozo y balsa': 402, 'AGUAS RESIDUALES': 362, 'RIO': 317, 'AGUAS RESIDUALES-BALSA': 268, 'RIO GUADALQUIVIR': 255, 'EMBALSE SAN RAFAEL DE NAVALLANA': 217, 'RIO GUADIANA MENOR': 200, 'PRESA': 199, 'RÍO GENIL': 179, 'ARROYO': 175, 'AGUAS RESIDUALES MUNICIPIO': 149, 'Comunidad de regantes Genil-Cabra': 143, 'Comunidad de regantes Tarajal': 123, 'ARROYO AGUA RESIDUAL': 112, 'SIN DEFINIR': 100, 'aguas depuradas': 89, 'Comunidad de regantes': 88, 'CANAL RUMBLAR': 81, 'GUADALQUIVIR': 77, 'RESIDUALES': 67, 'POZO PROPIO Y OTRA PARTE DEL RIO': 56, 'RIO GUADALQUIVIR/ BALSA': 54, 'LA LAGUNA': 53, 'E.D.A.R.': 51, 'Com. Regantes'

47. 209_riego:_calidad_del_agua

In [60]:
#This field records the quality of the water
#The quality needs to be homogenous ("APTAS PARA EL RIEGO','Aptas para riego','APTAS PARA RIEGO'") and is rather unspecific ("UTILIZABLES CON PRECAUCIONES")

print(feature_frame["209_riego:_calidad_del_agua"].isnull().sum())
counter_calidadagua=Counter(feature_frame["209_riego:_calidad_del_agua"])
print(counter_calidadagua)

427275
Counter({nan: 427275, 'APTAS PARA EL RIEGO': 47967, 'Aptas para riego': 47893, 'APTAS PARA RIEGO': 3008, 'UTILIZABLES CON PRECAUCIONES': 2541, 'APTA PARA RIEGO': 1761, 'Aptas para riogo': 850, 'BUENA': 817, 'Utilizable con precauciones': 781, 'No aptas para riego': 713, '-': 701, 'OPTIMA': 505, 'APTA': 414, 'MEDIANA CALIDAD': 270, 'BUENA- AGUA DE LLUVIA': 217, 'De dudosa a no válida': 197, 'REGULAR-VERTICILOSIS, SALINIDAD, PARTÍCULAS SUSPEN': 179, 'SIN DEFINIR': 139, 'UTILIZABLE CON PRECAUCIONES': 97, 'buena': 35, '0': 31, 'BUENA-POTABLE': 26, 'De buena a mediana': 25, 'Apta para riego': 2})


48. 313_variedad_secundaria

In [61]:
#This field records the secondary variant of olive tree in a landplot / measurement
#It requires homogenizing, as some components are repeated using different names: "Picual, Marteño", "PICUAL / MARTEÑO", "PICUAL", for instance.
#The data is quite unspecific, there are no % that indicate how common the subvariety is, so maybe it is not that relevant.

print(feature_frame["313_variedad_secundaria"].isnull().sum())
counter_variedadsecundaria=Counter(feature_frame["313_variedad_secundaria"])
print(counter_variedadsecundaria)

454577
Counter({nan: 454577, 'Picual, Marteño': 11206, 'Hojiblanca, Lucentino': 8325, 'HOJIBLANCO': 6423, 'PICUDO': 5513, 'PICUAL / MARTEÑO': 5460, 'Picudo': 5083, 'MANZANILLO': 5073, 'Manzanilla de Sevilla': 4324, 'Nevadillo Negro': 3989, 'Lechín de Sevilla, Zorzaleño, Ecijano': 3568, 'PICUAL': 3293, 'NEVADILLO BLANCO': 2977, 'Arbequina': 2118, 'ARBEQUINA': 1743, 'Gordal Sevillana': 1681, 'LECHIN': 1668, 'Lucio': 1116, 'MANZANILLO DE JAEN': 921, 'Verdial de Huévar': 920, 'NINGUNA': 660, 'VERDIAL': 536, 'Verdial de Vélez-Málaga': 475, 'Morona': 432, 'CARRASQUEÑO': 416, 'HOJIBLANCA': 329, 'Loaime': 279, 'PICUAL/ MARTEÑO (15000 OLIVOS)': 267, 'GORDAL': 266, 'Ocal': 239, 'Royal de Cazorla': 212, 'PAJARERO': 202, 'LECHIN (300 OLIVOS)': 199, 'ALOREÑA': 180, 'PICUDO Y MARTEÑO': 160, 'NO': 155, 'Manzanilla del Centro': 146, 'SIN ASIGNAR': 139, 'MARTEÑA': 127, 'Aloreña': 117, 'Frantoio': 107, 'Cornicabra': 95, 'Lechín de Granada, Cuquillo': 89, 'Koroneiki': 75, 'ROYAL': 73, 'Gordal de Granada'

49. 107_zona_homogénea

In [62]:
#This field apparently defines some subarea or denomination of where the landplot is
#Some of them may be specific landplots, approximate locations, RAIF biological zones (next variable), some look like routes...
#Most probably other geographic variables are more important or illustrative

print(feature_frame["107_zona_homogénea"].isnull().sum())
counter_zonahomogenea=Counter(feature_frame["107_zona_homogénea"])
print(counter_zonahomogenea)

204781
Counter({nan: 204781, 'GR/OL/07': 32299, 'GR/OL/01': 31782, 'CO/OL/10': 10297, 'JA012': 9139, 'OLIVAR DE SIERRA': 8765, 'GR/OL/02': 8593, 'CO/OL/09': 7645, 'JA/OL/12': 7036, 'CO/OL/08': 6651, 'CA/OL/04': 6615, 'ZONA 5': 5987, 'GR/OL/07 IZNALLOZ': 4639, '2305': 4504, 'GR/OL/03': 4129, 'CO/OL/07': 3992, 'CO/OL/06': 3338, 'CO/OL/11': 3325, 'CO/OL/13': 3098, 'GR/OL/08': 3058, 'CO006': 2859, 'MA/OL/04': 2788, 'H': 2748, 'OLIVAR DE CAMPIÑA': 2537, 'CA/OL/05': 2232, 'SE/OL/04': 2153, 'SE/OL/02': 2110, 'JA010': 1875, 'JA/OL/02': 1744, '13': 1621, 'CO/OL/02': 1594, 'ANTEQUERA NORTE': 1592, 'CA/OL/03': 1540, 'JA002': 1514, 'AL/OL/03': 1472, 'HU/OL/02': 1404, 'GR/OL/09': 1401, 'GR/OL/04': 1371, 'CO/OL/12': 1363, 'ANTEQUERA': 1363, 'Zona 5': 1289, '14': 1217, 'JA/OL/13': 1180, 'HU/OL/03': 1163, 'SE/OL/01': 1132, 'JA/OL/11': 1107, 'HU/OL/01': 1101, 'GR/OL/06': 1092, 'CO/OL/01': 1090, 'I': 1077, 'Estudio Jaén': 1031, 'ESTUDIO JAEN': 1022, 'CO/OL/05': 1009, 'JA/OL/01': 869, 'LOMA BAJA': 865, '

50. 120_zona_biológica_raif

In [63]:
#This field defines the biological zone inside the sanitary and plague responses network in Andalucia (Red de Alerta e Información Fitosanitaria)
#It is addressed not only at olive trees, but at other crops
#https://www.juntadeandalucia.es/servicios/sede/tramites/procedimientos/detalle/14776.html
#https://www.juntadeandalucia.es/agriculturapescaaguaydesarrollorural/raif/

#The origin of this database may be https://www.juntadeandalucia.es/datosabiertos/portal/dataset/raif/resource/74062bbf-8391-460b-97c3-3aec55be5d77,
#or this one may be more complete, maybe worth checking if some nulls here are not reflected there.


print(feature_frame["120_zona_biológica_raif"].isnull().sum())
counter_zonabiologica=Counter(feature_frame["120_zona_biológica_raif"])
print(counter_zonabiologica)

7303
Counter({'GR/OL/07 IZNALLOZ': 48820, 'JA/OL/12 LOMA BAJA': 44075, 'MA/OL/04 ANTEQUERA-NORTE': 43553, 'GR/OL/01 MONTEFRIO': 40106, 'CO/OL/10 CAMPIÑA ALTA OCCIDENTAL': 27797, 'JA/OL/08 SIERRA AHILLOS Y SAN PEDRO': 23694, 'GR/OL/02 LA VEGA': 19451, 'CO/OL/08 CAMPIÑA ALTA ORIENTAL I': 14511, 'CO/OL/06 CAMPIÑA BAJA OCCIDENTAL': 13483, 'MA/OL/07 ANTEQUERA OCCIDENTAL': 12009, 'JA/OL/02 LOMA ALTA': 11890, 'CO/OL/02 SIERRA MORENA CENTRAL': 11426, 'JA/OL/13 CAMPIÑA SUR': 10471, 'CO/OL/09 CAMPIÑA ALTA ORIENTAL II': 10390, 'SE/OL/01 ESTEPA NORTE': 9625, 'GR/OL/03 ALHAMA DE GRANADA': 9262, 'JA/OL/01 SIERRA MORENA': 8716, 'SE/OL/08 CAMPIÑA': 8551, 'JA/OL/05 SIERRA SUR': 8466, 'JA/OL/15 SIERRA CAZORLA': 8067, nan: 7303, 'CO/OL/11 SUBBÉTICA SEPTENTRIONAL': 7192, 'JA/OL/14 CAMPIÑA NORTE': 7079, 'CA/OL/04 OLVERA': 6968, 'JA/OL/06 SIERRA SEGURA': 6673, 'CO/OL/04 SIERRA MORENA OCCIDENTAL': 6646, 'SE/OL/02 ESTEPA SUR': 6515, 'CO/OL/05 LAS COLONIAS-VEGA BAJA': 6346, 'CO/OL/03 SIERRA MORENA GUADIATO': 5

51. 401_estación_climática_asociada

In [64]:
#This field defines the weather station attributed/closest to the landplot.
#More info on weather stations of Andalucia https://helvia.uco.es/bitstream/handle/10396/22714/tfm_laura_rodriguez_navarro.pdf
#In theory, most weather stations are from RAIF organisation, the same one of the biological zones.
#It needs homogeneizing too, e.g. maybe "JA012" is referred as "JAO12", or BAEZA09

print(feature_frame["401_estación_climática_asociada"].isnull().sum())
counter_estacionclimatica=Counter(feature_frame["401_estación_climática_asociada"])
print(counter_estacionclimatica)

156637
Counter({nan: 156637, 'JA012': 34414, 'GR001': 31274, 'GR006': 28849, 'JA008': 24438, 'CO008': 12619, 'CO006': 8314, 'CO004': 8215, 'JA013': 8017, 'CO011': 7349, 'SE011': 6952, 'JA002': 6817, 'CA003 - OLVERA': 6579, 'GR008': 6560, 'CO001': 6422, 'CO 001': 6352, 'JA001': 6007, 'JA006': 5501, 'CA003': 5204, 'JA015': 5047, 'Campillos': 4959, 'JA010': 4838, 'JA004': 4519, 'CO005': 4249, 'SE005': 4137, 'JA005': 4102, 'JA014': 4087, 'JA009': 3964, 'CO007': 3958, 'SE010': 3736, 'ADAMUZ': 3343, 'GR004': 3297, 'CA 005': 3275, 'IZNALLOZ': 3268, 'SIERRA YEGUAS': 2947, 'Archidona': 2934, 'AEROPUERTO': 2797, 'JA011': 2754, 'MA005': 2740, 'La Rambla': 2627, 'EL SAUCEJO': 2606, 'Villanueva de Algaida': 2539, 'Osuna': 2536, 'CO009': 2197, 'MA008': 1964, 'HU004': 1800, 'Estepa': 1791, 'Sierra Yeguas': 1760, 'GR006 Montefrio': 1723, 'GR009': 1650, 'COOO7': 1580, 'GR002': 1547, 'Santaella': 1524, 'Antequera': 1482, 'CORDOBA': 1473, 'GR005': 1423, 'CO010': 1371, 'BELMEZ': 1328, 'El Saucejo': 1273, 

52. 402_sensor_climático_asociado

In [65]:
#This field has more null values than the previous case, maybe defines
#the sensor in the weather station if it has more than 1 sensor.
#It should not be relevant.

print(feature_frame["402_sensor_climático_asociado"].isnull().sum())
counter_sensorclimatico=Counter(feature_frame["402_sensor_climático_asociado"])
print(counter_sensorclimatico)

379136
Counter({nan: 379136, 'GR006': 12048, 'JA012': 11757, 'GR001': 6993, 'CA003 - OLVERA': 6530, 'SE011': 5974, 'CO006': 5529, 'CO011': 4966, 'Campillos': 4918, 'CA003': 4144, 'HUELMA': 4041, 'SE005': 3840, 'BEDMAR': 3822, 'CO001': 3426, 'ADAMUZ': 3343, 'CA 005': 3275, 'JA015': 3123, 'LOS VILLARES': 2903, 'SIERRA YEGUAS': 2862, 'Osuna': 2444, 'JA013': 2421, 'JA002': 2342, 'JA010': 2037, 'AEROPUERTO': 1899, 'IZNALLOZ': 1870, 'La Rambla': 1806, 'CO004': 1523, 'CO008': 1519, 'CORDOBA': 1473, 'Archidona': 1381, 'El Saucejo': 1342, 'BELMEZ': 1328, 'TABERNAS': 1319, 'SE012': 1262, 'Santaella': 1244, 'CO 010': 1232, 'COOO5': 1191, 'CO005': 1180, 'GR008': 1117, 'AL008': 1110, 'BAEZA': 1109, 'Loja': 1073, 'Iznalloz': 1072, 'GR002': 1041, 'JA014': 951, 'JA009': 931, 'JA011': 913, 'CO 002': 910, 'Villanueva de Algaida': 845, 'JA001': 825, 'CO007': 736, 'SANTISTEBAN DEL PUERTO': 723, 'OSUNA': 710, 'Lucena': 704, 'Estepa': 701, 'JA005': 687, 'MA-08 Campillos': 655, 'Sierra Yeguas': 609, 'JA006':

53. 207_riego:_sistema_usual_de_riego

In [66]:
#This field has again many null values, and refers to how the olive trees are watered
#It needs homogeneizing too, e.g. "Localizado gotero", "LOCALIZADO GOTERO", null values may be "ninguno" (after all, all secano olive trees should be "ninguno")...
#It seems like a relevant variable

print(feature_frame["207_riego:_sistema_usual_de_riego"].isnull().sum())
counter_sistemariego=Counter(feature_frame["207_riego:_sistema_usual_de_riego"])
print(counter_sistemariego)

368102
Counter({nan: 368102, 'Localizado gotero': 83922, 'LOCALIZADO GOTERO': 52483, 'NINGUNO': 4970, 'goteo': 4292, 'LOCALIZADO MICROASPERSOR': 3283, 'GOTEO': 2243, 'Gravedad manta': 2068, 'LOCALIZADO LATIGUILLO': 1986, 'No existe sistema': 1319, 'GRAVEDAD MANTA': 1216, 'Localizado microaspersor': 779, 'Gravedad pies': 734, 'RIEGO POR GOTEO': 718, 'Sin riego': 701, 'GRAVEDAD SURCOS': 609, 'SUBTERRANEO': 537, 'Localizado': 490, 'GRAVEDAD PIES': 471, 'Goteo': 456, 'LOCALIZADO POR GOTEO': 439, 'Integrado': 417, 'RIEGO GOTEO': 414, 'Aspersores': 399, 'Riego por aspersión': 362, 'LOCALIZADO GOTERO SUPERFICIAL': 356, 'LOCALIZADO MICRODIFUSOR': 253, 'ASPERSORES': 218, 'localizado tuberia exudacion': 212, 'Riego por goteo': 196, 'SOTERRADO': 195, 'Localizado gotero interlinea cada 3 metros': 172, 'SECANO': 148, 'RANGER': 128, 'MEDIANTE CUBA': 111, 'SIN DEFINIR': 100, 'LOCALIZADO GOTERO SUBTERRÁNEO': 96, 'RIEGO OCASIONAL': 96, 'localizado': 91, 'ASPERSIÓN': 83, 'Localizado subterráneo': 76, 'L

54. 108_u_h_c_a_la_que_pertenece

In [67]:
#This field refers to some kind of weather area parameter, but is a bit redundant with other variables.

print(feature_frame["108_u_h_c_a_la_que_pertenece"].isnull().sum())
counter_uhca=Counter(feature_frame["108_u_h_c_a_la_que_pertenece"])
print(counter_uhca)

250324
Counter({nan: 250324, 'GR01': 9161, 'D': 4011, '3': 2576, '1': 2463, '2': 2117, '026': 2055, 'E': 1990, '10': 1985, '4': 1983, '094': 1893, '8': 1717, '7': 1661, '9': 1602, '5': 1596, '6': 1536, '11': 1453, '018': 1315, '025': 1289, 'UHC 1': 1217, 'UHC 3': 939, 'MOLINA DE SAN FERNANDO-GUADALBARBO': 921, '12': 915, 'UHC-4': 894, 'HU004': 855, 'CALATRAVEÑO': 852, 'ARAMA': 847, 'UHC-1': 843, '13': 826, 'UHC 3: CAMPIÑA ESTE': 786, 'UHC-2': 738, 'LOS QUEMAOS-MOLINETA-GACHERO': 719, 'RETAMALES': 683, 'ZONA 3 OLVERA': 669, 'UHC 5: SIERRA SUR': 650, 'FUENTE HIGUERA': 628, 'DARRO': 624, '099': 619, 'UHC-5': 617, '14': 610, 'UHC2': 608, 'UHC 1: RIEGO PICUAL MAYOR': 600, 'LA VEGA': 595, 'CORTIJO NUEVO': 593, 'MEDIA LEGUA': 590, 'HU007': 580, 'UHC 4': 563, 'U.H.C. 1': 559, 'UHC 2: CAMPIÑA OESTE': 553, 'BAQUETA': 552, 'LOS LLANOS': 543, 'CORTIJUELO': 538, 'EL PALOMAR': 530, 'UHC Nº 3': 527, 'UHC Nº 13': 525, 'UHC Nº 2': 523, 'UHC 6: ALBENDÍN-LUQUE': 514, 'LOS TIRAOS': 513, 'UHC Nº 9': 512, '

55. 316_fecha_de_plantación_variedad_secundaria

In [68]:
#This field refers to the plantation date of the secondary variety of olive tree, if it exists.
#Not all entries have this second variety and most fields are null, this variable is probably not relevant for the model.

print(feature_frame["316_fecha_de_plantación_variedad_secundaria"].isnull().sum())
counter_fechaplantacion=Counter(feature_frame["316_fecha_de_plantación_variedad_secundaria"])
print(counter_fechaplantacion)

516075
Counter({nan: 516075, '01/03/1990': 1384, '01/03/1995': 1297, '25/05/1900': 1096, '01/03/1960': 864, '01/03/1910': 729, '1996': 652, '01/03/1920': 609, '1990': 599, '2002': 482, '01/03/2000': 445, '1850': 418, '01/03/1996': 416, '01/03/1950': 392, '1995': 347, '01/03/1992': 340, '01/03/1900': 331, '1900': 317, '08/03/1926': 303, '01/03/1998': 302, '05/01/1935': 301, '2000': 294, '01/03/1993': 286, '01/03/1980': 282, '01/03/1985': 269, '1971': 266, '01/05/1970': 262, '1992': 261, '01/03/1997': 246, '01/05/1910': 245, '1885': 244, '1974': 226, '2005': 208, '01/01/1996': 206, '1940': 204, '1975': 199, '01/03/2009': 189, '01/01/1940': 180, '01/03/1999': 174, '06/06/1996': 166, '14/11/2003': 162, '25/05/1920': 161, '15/05/1900': 158, '01/01/1995': 157, '01/03/1988': 148, '01/01/1990': 143, '03/02/1900': 138, '01/10/1990': 138, '01/03/2005': 137, '01/01/2000': 130, '11/11/1996': 123, '01/10/2012': 118, '1985': 113, '04/04/1986': 109, '1999': 109, '01/03/1968': 108, '01/01/1997': 108, 

56. 315_patrón_variedad_secundaria

In [69]:
#This field appears to refer to the "pattern" of the secondary variant of olive.
#What might be the difference with 48)313_variedad_secundaria?
#No idea, but does not look like relevant.
#https://herograespeciales.com/etapas-del-ciclo-del-olivo/
#https://www.cultifort.com/fructificacion-produccion-maduracion-la-aceituna/
#https://www.caseriadehuellar.com/paginas/Noticia.asp?idNoticia=2104
#https://nevianfertilizantes.com/blog/factores-que-influyen-en-la-produccion-del-olivo/

print(feature_frame["315_patrón_variedad_secundaria"].isnull().sum())
counter_patronvariedad=Counter(feature_frame["315_patrón_variedad_secundaria"])
print(counter_patronvariedad)

519433
Counter({nan: 519433, 'Ninguno': 3177, 'NINGUNO': 2714, 'Lechin de Sevilla': 2214, 'HOJIBLANCA': 1977, 'Acebuche': 1914, 'Picual': 1333, 'Manzanilla': 1179, 'Hojiblanca': 743, 'MANZANILLO DE JAEN': 637, 'PICUAL': 352, 'SIN DEFINIR': 139, 'LUCIO': 114, 'ACEBUCHE': 109, 'Arbequina': 96, 'Verdial de Huevar': 82, 'NEVADILLO': 69, 'NINGUNA': 67, '-': 31, 'LECHÍN DE SEVILLA': 24, 'MANZANILLO': 20, 'LECHIN': 19, 'PICUAL O MARTEÑA': 1})


57. 317_%_superficie_ocupada_variedad_secundaria

In [70]:
#This field refers to the % of surface occupied by a second variety.
#The null values here are larger than the ones in 48. 313_variedad_secundaria, so for the majority
#of fields with a second variety of olive we do not have this percentage.
#Maybe, due to this quantities, this may be disregarded.


print(feature_frame["317_%_superficie_ocupada_variedad_secundaria"].isnull().sum())
feature_frame["317_%_superficie_ocupada_variedad_secundaria"].describe()


515049


count    21395.000000
mean        23.256321
std         15.813231
min          0.000000
25%         10.000000
50%         25.000000
75%         34.000000
max         60.000000
Name: 317_%_superficie_ocupada_variedad_secundaria, dtype: float64

58. 306_altura_de_copa_(m)

In [71]:
#This field refers to the height of the olive tree.
#Most values are null, and the are some values clearly incorrect (mean and maximum value are completely incorrect)
#It may be relevant.

print(feature_frame["306_altura_de_copa_(m)"].isnull().sum())
feature_frame["306_altura_de_copa_(m)"].describe()

468536


count    6.790800e+04
mean     1.128841e+05
std      1.418981e+06
min      0.000000e+00
25%      3.000000e+00
50%      3.000000e+00
75%      3.500000e+00
max      1.803572e+07
Name: 306_altura_de_copa_(m), dtype: float64

59. 310_patrón_variedad_principal

In [72]:
#Again, the variable "pattern" of the olive variety is a bit misleading.
#What might be the difference with 308_variedad_principal?
#Maybe it is worth comparing if the null values from there are the ones present here and the other way round, for some reason...
#No idea.


print(feature_frame["310_patrón_variedad_principal"].isnull().sum())
counter_patronvariedadprincipal=Counter(feature_frame["310_patrón_variedad_principal"])
print(counter_patronvariedadprincipal)

410071
Counter({nan: 410071, 'Picual': 46345, 'PICUAL': 34192, 'Hojiblanca': 13183, 'Ninguno': 10588, 'NINGUNO': 8170, 'Acebuche': 5999, 'Lechin de Sevilla': 2564, 'Manzanilla': 1312, 'HOJIBLANCA': 1094, 'ACEBUCHE': 1016, 'NEVADILLO': 553, 'Arbequina': 431, 'Verdial de Huevar': 180, 'SIN DEFINIR': 139, 'LECHIN': 138, 'NO': 111, 'hojiblanco': 72, 'HOHIBLANCA': 71, 'NINGUNA': 67, 'ARBEQUINA': 44, 'hojiblanca': 31, 'I 18': 27, 'AUTOENRAIZADA (NO INJERTADO)': 26, 'PICO LIMON': 19, 'PICUAL O MARTEÑA': 1})


60. 411_representa_a_la_u_h_c_(si/no)

In [73]:
#This may be if the data is representative of the weather unit defined before?
#As we have directly all the raw data, maybe it is not worth on aggregating results / conclusions
#in these kind of invented subzones.


print(feature_frame["411_representa_a_la_u_h_c_(si/no)"].isnull().sum())
counter_representaalauhc=Counter(feature_frame["411_representa_a_la_u_h_c_(si/no)"])
print(counter_representaalauhc)

357609
Counter({nan: 357609, 'SI': 178696, 'NO': 139})


61. 109_sistema_para_el_cumplimiento_gestión_integrada

In [74]:
#This variable deals with the sustainability of the agricultural explotation.



print(feature_frame["109_sistema_para_el_cumplimiento_gestión_integrada"].isnull().sum())
counter_gestionintegrada=Counter(feature_frame["109_sistema_para_el_cumplimiento_gestión_integrada"])
print(counter_gestionintegrada)

400111
Counter({nan: 400111, 'Producción Integrada (PI)': 132330, 'Entidad de Asesoramiento (EA)': 3337, 'Otro sistema Asesorado (OA)': 523, 'Agricultura Ecológica (AE)': 86, 'Producción certificada (PC)': 57})
